# Scraper Code for Analytics for Unstructured data project

**Team members:**

1. Aditya Sindhavad
2. Adithya Murali
3. Advaith Shankar
4. Manasa Maganti
5. Shashank Rao
6. Varsha Manju Jayakumar

**Website Scraped** : https://www.fragrantica.com/

In [ ]:
import pandas as pd
import time
import random
from selenium import webdriver
from bs4 import BeautifulSoup

# Load the Excel file that contains the URLs
urls_df = pd.read_excel('url_listM.xlsx')  # Replace with the actual file name
urls = urls_df['url']  # Assuming the column in the Excel file is labeled 'URLs'

# Initialize a list to store the data
scraped_data = []

# Set up the WebDriver (e.g., Chrome)

# Function to scroll the page by random units and wait a random amount of time
def incremental_scroll(driver):
    last_position = driver.execute_script("return window.pageYOffset;")
    while True:
        # Scroll by a random number of pixels between 450 and 700
        scroll_amount = random.randint(450, 700)
        driver.execute_script(f"window.scrollBy(0, {scroll_amount});")
        
        # Wait for a random time between 1 and 4 seconds
        time.sleep(random.uniform(1, 4))

        new_position = driver.execute_script("return window.pageYOffset;")
        if new_position == last_position:
            break  # Break if the scroll position hasn't changed (end of page reached)
        last_position = new_position

# Loop through each URL, scrape the content, and store it in the list
for url in urls:
    try:
        # Open the website
        driver = webdriver.Chrome()  # Make sure you have the Chrome WebDriver in your PATH
        driver.get(url)

        # Scroll to the end of the page
        incremental_scroll(driver)

        # Parse the page with BeautifulSoup after scrolling is complete
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Append the URL and the soup content (as text) to the list
        scraped_data.append({'URL': url, 'HTML Content': str(soup)})
    
    except Exception as e:
        print(f"Failed to scrape {url}: {e}")

    # Close the WebDriver after all URLs have been scraped
    driver.quit()

# Create a DataFrame from the scraped data
scraped_df = pd.DataFrame(scraped_data)

# Optionally, save the DataFrame to a CSV file
#scraped_df.to_csv('scraped_websites_data.csv', index=False)

# Display the DataFrame
#print(scraped_df)


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

# Assuming scraped_df already contains the scraped HTML content in 'HTML Content' column
# with URLs in 'URL' column

# Prepare a list to store the extracted review data
review_data = []

# Loop over each row in the scraped_df DataFrame
for index, row in scraped_df.iterrows():
    try:
        # Get the HTML content and parse it with BeautifulSoup
        soup = BeautifulSoup(row['HTML Content'], 'html.parser')

        # Extract the page title
        page_title = soup.title.get_text() if soup.title else 'No Title'

        # Find all <div> elements with itemprop="reviewBody"
        review_divs = soup.find_all('div', itemprop='reviewBody')

        # Loop through all the review divs and store the text content along with the page title
        for div in review_divs:
            review_content = div.get_text(strip=True)  # Extract the review content text
            # Append the data as a dictionary to the list
            review_data.append({
                'URL': row['URL'],
                'Title': page_title,
                'Review': review_content
            })

    except Exception as e:
        print(f"Error processing URL {row['URL']}: {e}")

# Create a DataFrame from the review_data list
reviews_df = pd.DataFrame(review_data)

# Optionally, save the DataFrame to a CSV file
reviews_df.to_csv('extracted_reviews_data4.csv', index=False)

# Display the DataFrame
print(reviews_df)
